In [1]:
# import sys
# !{sys.executable} -m pip install pandas;

In [2]:
import pandas as pd
import time
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
df = pd.DataFrame(pd.read_csv("../logs/arbbot.csv"))
df["humanTimestamp"] = pd.to_datetime(df['blockTimestamp'], unit='s')
df["runTime"] = df["endTimestamp"] - df["startTimestamp"]

In [4]:
# Limit to hour span
hour_limit = 24
sec_limit = hour_limit * 3600
df = df[df.blockTimestamp > time.time()-sec_limit]

In [6]:
# df[df["netProfit"]>0].sort_values(by="netProfit", ascending=0)
df[df["blockNumber"]==11534932]

,blockNumber,blockTimestamp,instrName,instrId,optimalAmount,grossProfit,netProfit,gasCost,gasAmount,startTimestamp,endTimestamp,humanTimestamp,runTime


## Best opportunities so far

In [7]:
df.sort_values(by="netProfit", ascending=0).head(10)

,blockNumber,blockTimestamp,instrName,instrId,optimalAmount,grossProfit,netProfit,gasCost,gasAmount,startTimestamp,endTimestamp,humanTimestamp,runTime
15473,11546663.0,1609214945,weth2kp3r2weth_uniswap2Sushiswap,I0011,28.050,0.49260,0.46860,0.0240,200000,1.609215e+09,1.609215e+09,2020-12-29 04:09:05,0.300499
14017,11545916.0,1609204726,weth2kp3rweth_sushiswap2Uniswap,I0012,18.080,0.20790,0.19470,0.0132,200000,1.609205e+09,1.609205e+09,2020-12-29 01:18:46,0.296577
14011,11545914.0,1609204723,weth2kp3rweth_sushiswap2Uniswap,I0012,18.080,0.20790,0.18990,0.0180,200000,1.609205e+09,1.609205e+09,2020-12-29 01:18:43,0.303927
14014,11545915.0,1609204724,weth2kp3rweth_sushiswap2Uniswap,I0012,18.080,0.20790,0.18670,0.0212,200000,1.609205e+09,1.609205e+09,2020-12-29 01:18:44,0.359222
15137,11546501.0,1609212645,weth2snx2weth_uniswap2Sushiswap,I0013,20.600,0.12000,0.09999,0.0200,200000,1.609213e+09,1.609213e+09,2020-12-29 03:30:45,0.351915
15381,11546630.0,1609214611,weth2kp3r2weth_uniswap2Sushiswap,I0011,11.530,0.08218,0.06258,0.0196,200000,1.609215e+09,1.609215e+09,2020-12-29 04:03:31,0.438420
12497,11545191.0,1609194956,weth2link2weth_uniswap2Sushiswap,I0003,29.440,0.06505,0.04245,0.0226,200000,1.609195e+09,1.609195e+09,2020-12-28 22:35:56,0.337882
15278,11546578.0,1609213845,weth2inj2weth_sushiswap2Uniswap,I0020,4.359,0.06240,0.03240,0.0300,200000,1.609214e+09,1.609214e+09,2020-12-29 03:50:45,0.542458
10196,11541106.0,1609141553,weth2ampl2weth_uniswap2Sushiswap,I0017,8.992,0.04491,0.03151,0.0134,200000,1.609142e+09,1.609142e+09,2020-12-28 07:45:53,0.351334
10189,11541103.0,1609141526,weth2ampl2weth_uniswap2Sushiswap,I0017,8.992,0.04491,0.03071,0.0142,200000,1.609142e+09,1.609142e+09,2020-12-28 07:45:26,0.553807


## Statistics

In [8]:
# Limit opportuninities to the ones beyond the threshold to account for gas costs
net_profitable_opps = df[df.netProfit > 0]
opp_count = net_profitable_opps.blockNumber.count()
runtime_mean = net_profitable_opps.runTime.mean()
max_profit = net_profitable_opps.netProfit.max()
mean_profit = net_profitable_opps.netProfit.mean()
total_profit = net_profitable_opps.netProfit.sum()

In [9]:
print(f"Opportunitys count: {opp_count}")
print(f"Latency average: {runtime_mean:.2f} sec")
print(f"Max profit: {max_profit:.2f} ETH")
print(f"Average profit: {mean_profit:.2f} ETH")

Opportunitys count: 62
Latency average: 0.74 sec
Max profit: 0.47 ETH
Average profit: 0.03 ETH


## Instruction analysis

In [10]:
opp_comp_df = net_profitable_opps.groupby("instrName").agg({"blockNumber": "count", "netProfit": ["max", "sum", "mean"]})
opp_comp_df.columns = ["count", "max_profit", "summed_profit", "mean_profit"]
opp_comp_df.index.name = None
opp_comp_df.sort_values(by="mean_profit", ascending=0)

,count,max_profit,summed_profit,mean_profit
weth2kp3r2weth_uniswap2Sushiswap,3,0.468600,0.544510,0.181503
weth2kp3rweth_sushiswap2Uniswap,7,0.194700,0.583200,0.083314
weth2link2weth_uniswap2Sushiswap,1,0.042450,0.042450,0.042450
weth2ampl2weth_uniswap2Sushiswap,3,0.031510,0.089930,0.029977
weth2inj2weth_sushiswap2Uniswap,2,0.032400,0.035755,0.017877
weth2snx2weth_uniswap2Sushiswap,12,0.099990,0.181678,0.015140
weth2snx2weth_sushiswap2Uniswap,4,0.017360,0.040852,0.010213
weth2wbtc2weth_sushiswap2Uniswap,1,0.008177,0.008177,0.008177
weth2inj2weth_uniswap2Sushiswap,20,0.024040,0.150940,0.007547
weth2sushi2weth_uniswap2Sushiswap,2,0.011050,0.013911,0.006955


In [11]:
df2 = pd.DataFrame(pd.read_csv("../graveyard/arbbot_ver1.csv"))

In [12]:
df2.sort_values(by="netProfit", ascending=0).head(10)

,blockNumber,blockTimestamp,finishTimestamp,instrName,instrId,optimalAmount,grossProfit,netProfit,gasCost,gasAmount,startTimestamp,endTimestamp
2007,11532040,1609021665,1609021672,weth2snx2weth_sushiswap2Uniswap,I0014,1161.0,4.596,4.580,0.01612,260000.0,1.609022e+09,1.609022e+09
488,11533307,1609038390,1609038397,weth2inj2weth_sushiswap2Uniswap,I0020,368.7,4.239,4.224,0.01482,260000.0,1.609038e+09,1.609038e+09
448,11533109,1609035739,1609035751,weth2inj2weth_sushiswap2Uniswap,I0020,333.6,3.360,3.342,0.01898,260000.0,1.609036e+09,1.609036e+09
451,11533264,1609037826,1609037830,weth2inj2weth_sushiswap2Uniswap,I0020,316.3,3.074,3.052,0.02158,260000.0,1.609038e+09,1.609038e+09
450,11533263,1609037803,1609037827,weth2inj2weth_sushiswap2Uniswap,I0020,316.3,3.074,3.050,0.02340,260000.0,1.609038e+09,1.609038e+09
1289,11532358,1609025933,1609025974,weth2band2weth_uniswap2Sushiswap,I0015,356.1,2.715,2.705,0.01066,260000.0,1.609026e+09,1.609026e+09
1290,11532359,1609025972,1609025983,weth2band2weth_uniswap2Sushiswap,I0015,356.1,2.715,2.701,0.01430,260000.0,1.609026e+09,1.609026e+09
499,11533326,1609038575,1609038587,weth2inj2weth_sushiswap2Uniswap,I0020,288.9,2.619,2.608,0.01092,260000.0,1.609039e+09,1.609039e+09
37,11531937,1609020474,1609020483,weth2inj2weth_sushiswap2Uniswap,I0020,283.9,2.421,2.405,0.01664,260000.0,1.609020e+09,1.609020e+09
3149,11533227,1609037362,1609037402,weth2sushi2weth_uniswap2Sushiswap,I0007,611.8,2.352,2.339,0.01300,260000.0,1.609037e+09,1.609037e+09
